In [271]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Подготовка данных

В данной работе я буду пользоваться учебным набором данных, содержащим информацию об оценках фильмов, подробную информацию о данном датасете можно прочитать по ссылке: https://www.kaggle.com/datasets/prajitdatta/movielens-100k-dataset.

1) Датасет с информацией о фильмах:

In [272]:
movies = pd.read_csv('ml-100k/u.item', sep = '|', header = None, engine='python', encoding='latin1')
movies.columns = ['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown',
                  'action', 'adventure', 'animation', 'childrens', 'comedy', 'crime', 'documentary', 'drama',
                  'fantasy', 'noir', 'horror', 'musical', 'mystery', 'romance', 'scifi', 'thriller', 'war', 
                  'western']
movies.drop(columns = ['video_release_date', 'IMDb_URL', 'unknown', 'release_date'], inplace=True)
movies.head()

,movie_id,movie_title,action,adventure,animation,childrens,comedy,crime,documentary,drama,fantasy,noir,horror,musical,mystery,romance,scifi,thriller,war,western
0,1,Toy Story (1995),0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


2) Датасет с информацией о пользователях:

In [273]:
users = pd.read_csv('ml-100k/u.user', sep = '|', header = None, engine='python', encoding='latin1')
users.columns = ['user_id', 'age', 'gender', 'job', 'zip_code']
users.head()

,user_id,age,gender,job,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


3) Датасет с рейтингами фильмов от пользователей:

In [274]:
ratings = pd.read_csv('ml-100k/u.data', sep = '\t', header = None, engine='python', encoding='latin1')
ratings.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


## Построение бейзлайнов

Строить и обучать модели будем с помощью библиотеки **Surprise**. Подробную документацию данной библиотеки можно найти по ссылке: https://surprise.readthedocs.io/en/stable/getting_started.html.

In [275]:
from sklearn.model_selection import train_test_split
from surprise import Reader, Dataset, SVD, NormalPredictor, KNNBasic
from surprise.model_selection import cross_validate, GridSearchCV, RandomizedSearchCV


In [289]:
data = Dataset.load_from_file('ml-100k/u.data', reader= Reader())

- В библиотеке **Surprise** есть 4 доступные метрики: `RMSE`, `MAE`, `MSE` и `FCP`. Первые 3 - это всем знакомые классические метрики в задачах машинного обучения, а подробнее про последнюю можно почитать по ссылке: https://www.ijcai.org/Proceedings/13/Papers/449.pdf. Мы будем использовать классический подход и измерять качество модели с помощью метрики `RMSE`(конечно, использование данной метрики в реальности при построении правильных рекомендаций не совсем корректно, подробнее об этом рассказано в данном видео: https://www.youtube.com/watch?v=Te_6TqEhyTI&t=2817s, однако на данном этапе ограничимся все же ей).
- Также в данной библиотеке реализованы такие методы, как `GridSearchCV`, с помощью которого мы будем подбирать оптимальные гиперпараметры для моделей, и `cross_validate`, с помощью которого мы будем измерять качество моделей.

Приступим к построению моделей.

### 1) Random model 

Это простой алгоритм, который будет заполнять случайно таблицу с рейтингами, основываясь на наивном предположении, что распределение рейтингов в тренировочном сете - нормальное. У данной модели нет гиперпараметров, поэтому GridSearchCV здесь не нужен.

In [295]:
model_random = NormalPredictor()
model_random_results = cross_validate(model_random, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5251  1.5116  1.5193  1.5214  1.5127  1.5180  0.0052  
Fit time          0.11    0.14    0.13    0.13    0.15    0.13    0.01    
Test time         0.10    0.11    0.11    0.13    0.11    0.11    0.01    


### 2) User based collaborative filtering 

Для предсказания алгоритм берет пользователя, находит других пользователей, которые ставили схожие с ним рейтинги, и предлагает товары, которые понравились похожим пользователям(алгоритм работает на основе KNN). У данной модели есть основной гиперпараметр - количество соседей. Проведем поиск по сетке с помощью GridSearchCV и
найдем оптимальный гиперпараметр.

In [296]:
# написать поиск по сетке лучших гиперпараметров
param_grid = {'k': [10*i for i in range(1,10)], 'sim_options': {'user_based': [True]}}
gs = GridSearchCV(KNNBasic, param_grid, measures=['RMSE'], cv=3, n_jobs=-2)

gs.fit(data)

In [297]:
gs.best_params

{'rmse': {'k': 30, 'sim_options': {'user_based': True}}}

In [298]:
model_user_based = KNNBasic(k=30, sim_options={'user_based': True})
model_user_based_results = cross_validate(model_user_based, data, measures=['RMSE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9771  0.9810  0.9741  0.9690  0.9837  0.9770  0.0052  
Fit time          0.31    0.36    0.28    0.30    0.34    0.32    0.03    
Test time         2.27    2.63    2.37    2.44    2.32    2.41    0.12    


### 3) Item based collaborative filtering 

В отличии от предыдущего пункта, алгоритм фокусируется на конкретном пользователе и на товары, которые ему понравились, вследствие чего будет рекомендовать похожие на эти понравившиеся товары(опять же алгоритм основан на KNN). У данной модели есть основной гиперпараметр - количество соседей. Проведем поиск по сетке с помощью GridSearchCV и найдем оптимальный гиперпараметр.

In [300]:
param_grid = {'k': [10*i for i in range(1,10)], 'sim_options': {'user_based': [False]}}
gs = GridSearchCV(KNNBasic, param_grid, measures=['RMSE'], cv=3, n_jobs=-2)

gs.fit(data)

In [301]:
gs.best_params

{'rmse': {'k': 40, 'sim_options': {'user_based': False}}}

In [302]:
model_item_based = KNNBasic(k=40, sim_options={'user_based': False})
model_item_based_results = cross_validate(model_item_based, data, measures=['RMSE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9701  0.9829  0.9740  0.9757  0.9723  0.9750  0.0044  
Fit time          0.45    0.44    0.42    0.40    0.41    0.42    0.02    
Test time         2.89    2.90    2.85    2.86    2.92    2.88    0.03    


### 4) Matrix factorization

Подход сложный, алгоритм тут описывать не буду, скажу лишь, что он основан на сингулярном разложении матрицы рейтингов(SVD). Подробнее про это можно прочитать по ссылке: http://www.machinelearning.ru/wiki/images/7/79/2015_417_KhomutovNY.pdf. У данной модели очень большие степени свободы, то есть довольно много гиперпараметров. Мы рассмотрим только базовые: **n_epochs**(количество итераций алгоритма SGD), **biased**(будем ли мы учитывать свободный член), **lr_all**(learning rate для всех параметров), **reg_all**(коэффициент регуляризации для всех параметров). Так как мы будем выбирать довольно много гиперпараметров, вместо модуля `GridSearchCV` будем использовать модуль `RandomizedSearchCV` для экономии времени.

In [303]:
param_grid = {'n_epochs': [5*i for i in range(1,10)],
              'lr_all': [0.001*i for i in range(1,10)],
              'reg_all': [0.01*i for i in range(1,10)],
              'biased': [True, False]}
gs = RandomizedSearchCV(SVD, param_grid, measures=['RMSE'], cv=3, n_jobs=-2)

gs.fit(data)

In [304]:
gs.best_params

{'rmse': {'n_epochs': 30, 'lr_all': 0.007, 'reg_all': 0.09, 'biased': True}}

In [305]:
model_mat_fac = SVD(n_epochs=30, lr_all=0.007, reg_all=0.09, biased=True)
model_mat_fac_results = cross_validate(model_mat_fac, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9155  0.9164  0.9181  0.9135  0.9136  0.9154  0.0017  
Fit time          1.27    1.61    1.25    1.31    1.71    1.43    0.19    
Test time         0.11    0.11    0.11    0.12    0.12    0.11    0.00    


### 5) Сравнение моделей 

In [306]:
means = [round(model_random_results['test_rmse'].mean(),4),
         round(model_user_based_results['test_rmse'].mean(),4),
         round(model_item_based_results['test_rmse'].mean(),4), 
         round(model_mat_fac_results['test_rmse'].mean(),4)]

print("\t RMSE Means for each model\n")
print(pd.Series(means, ['Random','User-based', 'Item-based', 'Matrix factorization']))

	 RMSE Means for each model

Random                  1.5180
User-based              0.9770
Item-based              0.9750
Matrix factorization    0.9154
dtype: float64


Как и предполагалось, худшей по качеству оказалась модель `Random`, а лучше всех отработала сложная модель `Matrix factorization`.

## Создание метрик Diversity и Novelty

1) Теперь реализуем функции, которые будут подсчитывать Diversity и Novelty. В качестве Diversity будем использовать стандартную метрику ILD(Average Intra-List Distance), в качестве Novelty - стандартную метрику MIUF(Mean Inverse User Frequency). Подробно про данные метрики можно прочитать по ссылке: https://link.springer.com/content/pdf/10.1007/978-1-4899-7637-6_26.pdf. Подсчет данных метрик будет производиться с помощью библиотеки `recmetrics`, документация доступна по ссылке: https://pythonrepo.com/repo/statisticianinstilettos-recmetrics-python-recommender-systems.

In [308]:
def get_top_n(predictions, n=10):
    # First map the predictions to each user.
    top_n = dict()
    for uid, iid, true_r, est, details in predictions:
        current = top_n.get(uid, [])
        current.append((iid, movies.loc[int(iid)-1,'movie_title'],round(est,2)))
        top_n[uid] = current

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[2], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

2) Создадим функцию с помощью библиотеки `recmetrics`, которая будет подсчитывать метрику разнообразия для наших моделей:

In [309]:
from recmetrics import intra_list_similarity

In [310]:
def Diversity(top_n, n=10):
    predicted_matrix=[]
    for user in top_n:
        ids = []
        for rec in top_n[user]:
            ids.append(int(rec[0]))
        predicted_matrix.append(ids)
        return intra_list_similarity(predicted=predicted_matrix, feature_df=movies.drop(columns = 'movie_title'))
        

3) Создадим функцию с помощью библиотеки `recmetrics`, которая будет подсчитывать метрику новизны для наших моделей:

In [311]:
from recmetrics import novelty

In [312]:
def Novelty(top_n, n=10):
    predicted_matrix=[]
    for user in top_n:
        ids = []
        for rec in top_n[user]:
            ids.append(int(rec[0]))
        predicted_matrix.append(ids)
    
    
    ratings_dict = ratings.groupby('movie_id').agg({
        'user_id': 'count'
    })['user_id'].to_dict()
    
    return novelty(predicted = predicted_matrix, pop = ratings_dict, u = users.shape[0], n = n)[0]

4) Посчитаем построенные метрики на исследуемых моделях:

In [313]:
trainset = data.build_full_trainset()
testset = trainset.build_anti_testset()

In [315]:
models = [model_random, model_user_based, model_item_based, model_mat_fac]
model_names = ['Random model', 'User based model', 'Item based model', 'SVD_model']
for model, model_name in zip(models, model_names):
    model.fit(trainset)
    predictions = model.test(testset)
    top_10 = get_top_n(predictions, n=10)
    print('Модель:', model_name)
    print('Diversity =', Diversity(top_10))
    print('Novelty =', Novelty(top_10))
    print('\n\n')

Модель: Random model
Diversity = 0.9999714407137267
Novelty = 3.194833938335172



Computing the msd similarity matrix...
Done computing similarity matrix.
Модель: User based model
Diversity = 0.9999993315309315
Novelty = 9.306491368175067



Computing the msd similarity matrix...
Done computing similarity matrix.
Модель: Item based model
Diversity = 0.9999990235152896
Novelty = 8.046864277550805



Модель: SVD_model
Diversity = 0.9993676711978838
Novelty = 3.0350367816708905





**Вывод**: в плане разнообразия нельзя выделить ни одну из моделей: относительно метрики `Diversity` все показывают примерно одинаковые результаты; в плане новизны сильно выделяются KNN-based модели: относительно метрики `Novelty` они работают сильно лучше других представленных моделей.

## Полезные ссылки, использованные в данном отчете

- https://link.springer.com/content/pdf/10.1007/978-1-4899-7637-6_26.pdf
- https://www.ijcai.org/Proceedings/13/Papers/449.pdf
- https://surprise.readthedocs.io/en/stable/getting_started.html
- https://www.kaggle.com/datasets/prajitdatta/movielens-100k-dataset
- http://www.machinelearning.ru/wiki/images/7/79/2015_417_KhomutovNY.pdf
- https://arxiv.org/pdf/1202.1112.pdf